In [2]:
! pip install  google-cloud-bigquery

  Using cached google_cloud_bigquery-3.30.0-py2.py3-none-any.whl.metadata (7.9 kB)
  Using cached google_api_core-2.24.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_auth-2.38.0-py2.py3-none-any.whl.metadata (4.8 kB)
  Using cached google_cloud_core-2.4.3-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached google_resumable_media-2.7.2-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached googleapis_common_protos-1.69.2-py3-none-any.whl.metadata (9.3 kB)
  Using cached protobuf-6.30.1-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached grpcio-1.71.0-cp313-cp313-win_amd64.whl.metadata (4.0 kB)
  Using cached grpcio_status-1.71.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1_modules-0.4.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9-py3-none-any

In [5]:
! pip install kagglehub

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

print("Path to dataset files:", path)

d:\data pipeline\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\koppo\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2


In [38]:
from google.cloud import bigquery
import pandas as pd
import os

In [39]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/Users/koppo/Downloads/valued-ceiling-454014-a9-f6887630e113.json"

In [40]:
#environment variables
print(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])

C:/Users/koppo/Downloads/valued-ceiling-454014-a9-f6887630e113.json


In [ ]:
client = bigquery.Client()
csv_file = "./data/olist_customers_dataset.csv"
csv_file = "./data/olist_geolocation_dataset.csv"
csv_file = "./data/olist_order_items_dataset.csv"
csv_file = "./data/olist_order_payments_dataset.csv.csv"
csv_file = "./data/olist_order_reviews_dataset.csv"
csv_file = "./data/olist_orders_dataset.csv"
csv_file = "./data/olist_products_dataset.csv"
csv_file = "./data/olist_sellers_dataset.csvolist_products_dataset.csv"
csv_file = "./data/data\product_category_name_translation.csv.csv"
df = pd.read_csv(csv_file)

In [52]:
print(df.shape[0])

1000163


In [53]:

df.dropna(how="all", inplace=True)  # Remove fully empty records
df.drop_duplicates(inplace=True)  # Remove duplicate records
df.rename(columns={"customer_id": "CustomerID"}, inplace=True)  # Rename columns for consistency

print(df.head())  # Preview data


   geolocation_zip_code_prefix  geolocation_lat  geolocation_lng  \
0                         1037       -23.545621       -46.639292   
1                         1046       -23.546081       -46.644820   
2                         1046       -23.546129       -46.642951   
3                         1041       -23.544392       -46.639499   
4                         1035       -23.541578       -46.641607   

  geolocation_city geolocation_state  
0        sao paulo                SP  
1        sao paulo                SP  
2        sao paulo                SP  
3        sao paulo                SP  
4        sao paulo                SP  


In [54]:
print(df.shape[0])

738332


In [45]:
column_names = df.columns.tolist()
print(column_names)
import numpy as np
type_mapping = {
    str: "STRING",
    'int64': "INTEGER",
    np.int64: "INTEGER",
    float: "FLOAT",
    bool: "BOOLEAN"
}

data_types_dict = {
    col: type_mapping.get(type(df[col].iloc[0]), "UNKNOWN") for col in df.columns
}

print(data_types_dict)




['CustomerID', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']
{'CustomerID': 'STRING', 'customer_unique_id': 'STRING', 'customer_zip_code_prefix': 'INTEGER', 'customer_city': 'STRING', 'customer_state': 'STRING'}


In [46]:
table_id = "valued-ceiling-454014.student.olist_customers"  # ProjectID.Dataset.TableName
schema=[]
for  key,value in data_types_dict.items():
    schema.append(bigquery.SchemaField(key,value))

# schema = [
#         bigquery.SchemaField("CustomerID", "STRING"),
#         bigquery.SchemaField("customer_unique_id", "STRING"),
#         bigquery.SchemaField("customer_zip_code_prefix", "INTEGER"),
#         bigquery.SchemaField("customer_city", "STRING"),
#         bigquery.SchemaField("customer_state", "STRING"),
#     ]
print(schema)

[SchemaField('CustomerID', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('customer_unique_id', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('customer_zip_code_prefix', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('customer_city', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('customer_state', 'STRING', 'NULLABLE', None, None, (), None)]


In [47]:
dataset_id = "valued-ceiling-454014-a9.sample"  # Corrected Dataset ID

try:
    client.get_dataset(dataset_id)  # Check if dataset exists
    print(f"Dataset {dataset_id} already exists.")
except:
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = "asia-south1"  # Ensure correct location
    client.create_dataset(dataset, exists_ok=True)
    print(f"Dataset {dataset_id} created.")


Dataset valued-ceiling-454014-a9.sample already exists.


In [50]:
PROJECT_ID = "valued-ceiling-454014-a9"
DATASET_ID = "sample"
TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.olist_customers"
job_config = bigquery.LoadJobConfig(schema=schema, write_disposition="WRITE_TRUNCATE")

job = client.load_table_from_dataframe(df, TABLE_ID, job_config=job_config)
print("Data successfully loaded into BigQuery!")





ValueError: This method requires pyarrow to be installed